In [7]:
import ccxt
import pandas as pd
import numpy as np

# 创建Binance交易所实例
exchange = ccxt.binance()

# 获取BTC/USDT的历史K线数据，时间间隔为5分钟
symbol = 'BTC/USDT'
timeframe = '5m'
limit = 5000  # 可以根据需要调整数量
ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

# 转换为pandas DataFrame
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

In [9]:
# 提取周几信息
df['weekday'] = df['timestamp'].dt.day_name()

# 计算每日波动幅度
df['daily_range'] = df['high'] - df['low']

# 计算每周波动最大的一天
weekly_max = df.resample('W-Mon', on='timestamp').apply(lambda x: x.loc[x['daily_range'].idxmax()])

# 统计每周波动最大的一天是周几的频率
weekday_counts = weekly_max['weekday'].value_counts(normalize=True) * 100

print("每周波动最大的一天是周几的概率：")
print(weekday_counts)

每周波动最大的一天是周几的概率：
weekday
Friday    100.0
Name: proportion, dtype: float64


In [5]:
# 统计5分钟K线的波动幅度分布
df['5m_range'] = df['high'] - df['low']
range_distribution = df['5m_range'].describe()

print("5分钟K线波动幅度分布：")
print(range_distribution)

# 统计某个关键价格（例如移动均线）的突破情况
df['20_SMA'] = df['close'].rolling(window=20).mean()
df['breakout_above_20SMA'] = df['close'] > df['20_SMA']

# 计算突破20SMA后的涨跌幅度
df['returns'] = df['close'].pct_change()
breakout_returns = df.loc[df['breakout_above_20SMA'], 'returns'].mean()

print(f"突破20SMA后平均涨幅为: {breakout_returns:.2%}")

5分钟K线波动幅度分布：
count    500.000000
mean     108.155820
std      101.917989
min       11.910000
25%       50.162500
50%       78.430000
75%      132.990000
max      920.360000
Name: 5m_range, dtype: float64
突破20SMA后平均涨幅为: 0.03%


In [ ]:
import ccxt
import pandas as pd
import numpy as np

# 创建Binance交易所实例
exchange = ccxt.binance()

# 获取BTC/USDT的历史K线数据，时间间隔为5分钟
symbol = 'BTC/USDT'
timeframe = '5m'
limit = 1000  # 可以根据需要调整数量
ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

# 转换为pandas DataFrame
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# 提取日期信息
df['date'] = df['timestamp'].dt.date

# 计算每日的最高价和最低价
daily_highs = df.groupby('date')['high'].max()
daily_lows = df.groupby('date')['low'].min()

# 添加每日最高价和最低价信息
df['daily_high'] = df['date'].map(daily_highs)
df['daily_low'] = df['date'].map(daily_lows)

# 判断每根K线是否为当天最高价或最低价
df['is_daily_high_reached'] = df['high'] >= df['daily_high']
df['is_daily_low_reached'] = df['low'] <= df['daily_low']

# 判断截至当前K线，是否已经达到过当天的最高价或最低价
df['high_or_low_reached'] = df.groupby('date').apply(lambda x: x['is_daily_high_reached'].cummax() | x['is_daily_low_reached'].cummax()).reset_index(level=0, drop=True)

# 计算每根K线后当天最高价或最低价已经出现的概率
probability_by_candle = df.groupby(df.index % (24*12))['high_or_low_reached'].mean() * 100

print("每根5分钟K线后，已出现当天最高价或最低价的概率：")
print(probability_by_candle)